<a href="https://colab.research.google.com/github/Bosy-Ayman/Rawi-Vision/blob/main/AI/UCF_Crime_Pretrained_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import cv2
import numpy as np
from transformers import VideoMAEForVideoClassification
import torch
import cv2
import numpy as np
import torch.nn as nn
from huggingface_hub import hf_hub_download
import torch
from transformers import AutoModelForVideoClassification, AutoProcessor
import cv2
import numpy as np
import torch
from transformers import AutoModelForVideoClassification, AutoProcessor
import cv2
import numpy as np


#[1] First Model :VideoMae

 https://huggingface.co/OPear/videomae-large-finetuned-UCF-Crime

## Define class mapping


In [3]:
class_mapping = {
    "Abuse": 0, "Arrest": 1, "Arson": 2, "Assault": 3, "Burglary": 4,
    "Explosion": 5, "Fighting": 6, "Normal Videos": 7, "Road Accidents": 8,
    "Robbery": 9, "Shooting": 10, "Shoplifting": 11, "Stealing": 12, "Vandalism": 13
}
reverse_mapping = {v: k for k, v in class_mapping.items()}


## Load model


In [4]:
model_name = "OPear/videomae-large-finetuned-UCF-Crime"
device = "cuda" if torch.cuda.is_available() else "cpu"


In [5]:
model = VideoMAEForVideoClassification.from_pretrained(
    model_name,
    label2id=class_mapping,
    id2label=reverse_mapping,
    ignore_mismatched_sizes=True,
).to(device)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

VideoMAEForVideoClassification(
  (videomae): VideoMAEModel(
    (embeddings): VideoMAEEmbeddings(
      (patch_embeddings): VideoMAEPatchEmbeddings(
        (projection): Conv3d(3, 1024, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
    )
    (encoder): VideoMAEEncoder(
      (layer): ModuleList(
        (0-23): 24 x VideoMAELayer(
          (attention): VideoMAEAttention(
            (attention): VideoMAESelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=False)
              (key): Linear(in_features=1024, out_features=1024, bias=False)
              (value): Linear(in_features=1024, out_features=1024, bias=False)
            )
            (output): VideoMAESelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): VideoMAEIntermediate(
            (dense): Linear(in_features=1024, out_features=4096, bias=

## Video processing function


In [ ]:

def load_video_frames(video_path, num_frames=16, size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, size)
            frames.append(frame)

    cap.release()

    if len(frames) < num_frames:
        frames.extend([frames[-1]] * (num_frames - len(frames)))

    frames = np.stack(frames, axis=0)
    frames = torch.tensor(frames, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
    return frames

In [ ]:
video_path = "/content/young-woman-shoplifting-in-a-convenience-store.mp4"


In [ ]:

# Load and preprocess
video_tensor = load_video_frames(video_path)
video_tensor = video_tensor.unsqueeze(0).to(device)

# Inference
with torch.no_grad():
    outputs = model(video_tensor)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_label = torch.argmax(probs, dim=-1).item()

print(f"Predicted label: {reverse_mapping[predicted_label]}")

Predicted label: Burglary


# [2] Second Model : I3D

https://huggingface.co/Ahmeddawood0001/i3d_ucf_finetuned/blob/main/README.md

In [ ]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=e6e0371f3959911944ad558a077d6f73cf030341883f9c9a4ba7b90e92a22319
  Stored in directory: /root/.cache/pip/wheels/ed/9f/a5/e4f5b27454ccd4596bd8b62432c7d6b1ca9fa22aef9d70a16a
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=e28d043af235d9591d321f13bb61eeec335d165cda4601d8e1b573930a11bd23
  Stored in directory: /root/.cache/pip/wheels/7c/96/04/4f5f31ff812f684f69f40cb1634357812220aac58d4698048c
Successfully built fvcore iopath


## Define the model


In [ ]:
def load_i3d_ucf_finetuned(repo_id="Ahmeddawood0001/i3d_ucf_finetuned", filename="i3d_ucf_finetuned.pth"):
    class I3DClassifier(nn.Module):
        def __init__(self, num_classes):
            super(I3DClassifier, self).__init__()
            self.i3d = torch.hub.load('facebookresearch/pytorchvideo', 'i3d_r50', pretrained=True)
            self.dropout = nn.Dropout(0.3)
            self.i3d.blocks[6].proj = nn.Linear(2048, num_classes)
        def forward(self, x):
            x = self.i3d(x)
            x = self.dropout(x)
            return x
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = I3DClassifier(num_classes=8).to(device)
    weights_path = hf_hub_download(repo_id=repo_id, filename=filename)
    model.load_state_dict(torch.load(weights_path, map_location=device))
    model.eval()
    return model

## Define frame extraction function


In [ ]:
def extract_frames(video_path, max_frames=32, frame_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, frame_size)
        frames.append(frame)
    while len(frames) < max_frames:
        frames.append(frames[-1])
    frames = frames[:max_frames]
    frames = np.stack(frames)
    frames = torch.from_numpy(frames).permute(0, 3, 1, 2).float() / 255.0
    frames = frames.permute(1, 0, 2, 3)
    cap.release()
    return frames

## Define classification function


In [ ]:
def classify_video(video_path, model, labels):
    frames = extract_frames(video_path)
    frames = frames.unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(frames)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_idx = torch.argmax(probabilities, dim=1).item()
        predicted_label = labels[predicted_idx]
        confidence = probabilities[0, predicted_idx].item()
    return predicted_label, confidence


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
labels = ["arrest", "Explosion", "Fight", "normal", "roadaccidents", "shooting", "Stealing", "vandalism"]
model = load_i3d_ucf_finetuned()


In [ ]:
video_path = "/content/855260-hd_1920_1080_25fps.mp4"


In [ ]:

predicted_label, confidence = classify_video(video_path, model, labels)
print(f"Video: {video_path}")
print(f"Predicted Label: {predicted_label}")
print(f"Confidence: {confidence:.4f}")

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


Video: /content/855260-hd_1920_1080_25fps.mp4
Predicted Label: Stealing
Confidence: 0.3803


# [3] Third Model : It can only classify 2 labels (violent - non violent)

https://huggingface.co/Nikeytas/videomae-crime-detector-fixed-format

In [ ]:
!pip install transformers torch torchvision opencv-python pillow


## Load model and processor


In [ ]:
model = AutoModelForVideoClassification.from_pretrained("Nikeytas/videomae-crime-detector-fixed-format")
processor = AutoProcessor.from_pretrained("Nikeytas/videomae-crime-detector-fixed-format")


## Process video


In [ ]:
def classify_video(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise ValueError(f"Could not open video file: {video_path}")

    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        raise ValueError(f"Video {video_path} contains no frames.")

    # Generate indices to sample frames evenly
    indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)

    cap.release()

    if not frames:
        raise ValueError(f"No frames could be extracted from video {video_path}. It might be corrupted or unreadable.")

    if len(frames) < num_frames:
        frames.extend([frames[-1]] * (num_frames - len(frames)))

    frames = frames[:num_frames]

    inputs = processor(frames, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1).item()
        confidence = predictions[0][predicted_class].item()

    label = "Violent Crime" if predicted_class == 1 else "Non-Violent"
    return label, confidence

In [ ]:
video_path = "/content/7988417-uhd_2160_4096_25fps.mp4"


In [ ]:
prediction, confidence = classify_video(video_path)
print(f"Prediction: {prediction} (Confidence: {confidence:.3f})")

Prediction: Violent Crime (Confidence: 0.510)


#[4] Fourth model :Videamae ultra v1

**Issue:** only 2 labels

https://huggingface.co/Nikeytas/videomae-crime-detector-ultra-v1

In [ ]:
model = AutoModelForVideoClassification.from_pretrained("Nikeytas/videomae-crime-detector-ultra-v1")
processor = AutoProcessor.from_pretrained("Nikeytas/videomae-crime-detector-ultra-v1")

In [ ]:
def classify_video(video_path, num_frames=16):
    # Extract frames
    cap = cv2.VideoCapture(video_path)
    frames = []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)

    cap.release()

    # Process with model
    inputs = processor(frames, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1).item()
        confidence = predictions[0][predicted_class].item()

    label = "Violent Crime" if predicted_class == 1 else "Non-Violent"
    return label, confidence

In [ ]:
video_path = "/content/7988417-uhd_2160_4096_25fps.mp4"


In [ ]:
prediction, confidence = classify_video(video_path)
print(f"Prediction: {prediction} (Confidence: {confidence:.3f})")


Prediction: Violent Crime (Confidence: 0.993)
